In [1]:
from consumerComplaint.config.spark_manager import spark_session
from consumerComplaint.entity.schema import FinanceDataSchema
schema = FinanceDataSchema()

23/09/27 18:11:55 WARN Utils: Your hostname, Dell resolves to a loopback address: 127.0.1.1; using 192.168.29.77 instead (on interface wlp0s20f3)
23/09/27 18:11:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/suyodhan/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/suyodhan/.ivy2/cache
The jars for the packages stored in: /home/suyodhan/.ivy2/jars
com.amazonaws#aws-java-sdk added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8e056a32-5f10-4730-93f8-790a2e04dc77;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk;1.7.4 in central
	found commons-logging#commons-logging;1.1.1 in central
	found org.apache.httpcomponents#httpclient;4.2 in central
	found org.apache.httpcomponents#httpcore;4.2 in central
	found commons-codec#commons-codec;1.3 in central
	found com.fasterxml.jackson.core#jackson-core;2.1.1 in central
	found com.fasterxml.jackson.core#jackson-databind;2.1.1 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.1.1 in central
	found joda-time#joda-time;2.12.5 in central
	[2.12.5] joda-time#joda-time;[2.2,)
	found org.apache.hadoop#hadoop-aws;2.7.3 in central
	found org.apache.hadoop#hadoop-common;2.7.3 in 

In [5]:
import os
import sys
import joblib
import dill
from dataclasses import dataclass
from typing import List
from pyspark.ml.feature import StandardScaler, VectorAssembler, OneHotEncoder, StringIndexer, Imputer
from pyspark.ml.pipeline import Pipeline
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, rand

from consumerComplaint.utils.main_utils import save_object
from consumerComplaint.config.spark_manager import spark_session
from consumerComplaint.entity.schema import FinanceDataSchema
from consumerComplaint.entity.artifact_entity import DataValidationArtifact, DataTransformationArtifact
from consumerComplaint.entity.config_entity import DataTransformationConfig
from consumerComplaint.exception import ConsumerComplaintException
from consumerComplaint.logger import logger
from consumerComplaint.ml.feature import FrequencyImputer, DerivedFeatureGenerator
from pyspark.ml.feature import IDF, Tokenizer, HashingTF

In [2]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("my-app").getOrCreate()


In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession

# Create a SparkSession
# spark = SparkSession.builder.appName("pipeline-example").getOrCreate()

# Sample DataFrame
data = [(1, "A", 0.1), (2, "B", 0.2), (3, "C", 0.3)]
columns = ["feature1", "feature2", "label"]
df = spark_session.createDataFrame(data, columns)

# Define your pipeline stages (for demonstration purposes)
vector_assembler = VectorAssembler(inputCols=["feature1", "feature2"], outputCol="features")
pipeline = Pipeline(stages=[vector_assembler])

# Fit the pipeline to your DataFrame
trained_pipeline = pipeline.fit(df)

# Specify the directory where you want to save the trained pipeline
export_pipeline_dir = "pipeline"


# Save the trained pipeline to the specified directory
trained_pipeline.write().save(export_pipeline_dir)


# Load the saved pipeline
# loaded_pipeline = Pipeline.load(export_pipeline_dir)

# # Use the loaded pipeline to transform data (for demonstration purposes)
# transformed_data = loaded_pipeline.transform(df)

# # Show the transformed data
# transformed_data.show()


Py4JError: An error occurred while calling None.org.apache.spark.api.python.PythonFunction. Trace:
py4j.Py4JException: Constructor org.apache.spark.api.python.PythonFunction([class [B, class java.util.HashMap, class java.util.ArrayList, class java.lang.String, class java.lang.String, class java.util.ArrayList, class org.apache.spark.api.python.PythonAccumulatorV2]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:180)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:197)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:1589)



In [6]:
schema =  FinanceDataSchema()

In [7]:
stages = []

# numerical column transformation

# generating additional columns
derived_feature = DerivedFeatureGenerator(
    inputCols=schema.derived_input_features,
    outputCols=schema.derived_output_features
)
stages.append(derived_feature)
# creating imputer to fill null values
imputer = Imputer(
    inputCols=schema.numerical_columns,
    outputCols=schema.im_numerical_columns,
    strategy="mean"  # Modify the strategy as needed
)
stages.append(imputer)

frequency_imputer = FrequencyImputer(
    inputCols=schema.one_hot_encoding_features,
    outputCols=schema.im_one_hot_encoding_features
)
stages.append(frequency_imputer)

for im_one_hot_feature, string_indexer_col in zip(
    schema.im_one_hot_encoding_features,
    schema.string_indexer_one_hot_features
):
    string_indexer = StringIndexer(
        inputCol=im_one_hot_feature,
        outputCol=string_indexer_col
    )
    stages.append(string_indexer)

one_hot_encoder = OneHotEncoder(
    inputCols=schema.string_indexer_one_hot_features,
    outputCols=schema.tf_one_hot_encoding_features
)
stages.append(one_hot_encoder)

tokenizer = Tokenizer(
    inputCol=schema.tfidf_features[0],
    outputCol="words"
)
stages.append(tokenizer)

hashing_tf = HashingTF(
    inputCol=tokenizer.getOutputCol(),
    outputCol="rawFeatures",
    numFeatures=40
)
stages.append(hashing_tf)

idf = IDF(
    inputCol=hashing_tf.getOutputCol(),
    outputCol=schema.tf_tfidf_features[0]
)
stages.append(idf)

vector_assembler = VectorAssembler(
    inputCols=schema.input_features,
    outputCol=schema.vector_assembler_output
)
stages.append(vector_assembler)

standard_scaler = StandardScaler(
    inputCol=schema.vector_assembler_output,
    outputCol=schema.scaled_vector_input_features
)
stages.append(standard_scaler)


{'inputCols': ['company_response', 'consumer_consent_provided', 'submitted_via'], 'outputCols': ['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via']}


In [8]:
pipeline = Pipeline(
    stages=stages
)

In [10]:
file_path = "/home/suyodhan/Documents/Data-Science-Project/Consumer-Complaint-Dispute-Prediction/consumer_artifact/data_validation/20230927_152929/accepted_data/consumer_complaint"
dataframe: DataFrame = spark_session.read.parquet(file_path)
dataframe.printSchema()
# return dataframe

root
 |-- company: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- consumer_disputed: string (nullable = true)
 |-- date_received: string (nullable = true)
 |-- date_sent_to_company: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- product: string (nullable = true)
 |-- state: string (nullable = true)
 |-- sub_issue: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- timely: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [11]:
trained_pipeline = pipeline.fit(dataframe)

{'inputCols': ['company_response', 'consumer_consent_provided', 'submitted_via'], 'outputCols': ['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via']}


In [23]:
trained_pipeline

PipelineModel_a329f4d7b902

In [26]:
# trained_pipeline.write().save("/tmp/model")

In [15]:
# trained_pipeline.write().save("prashant")

In [19]:
# import dill

# file_name = "transformed_pipeline.joblib"  # Change this to your desired file name

# # Create the full file path for saving the pipeline
# # pipeline_file_path = "prashant"/file_name

# # Serialize and save the PySpark pipeline using dill
# with open(file_name, "wb") as file_obj:
#     dill.dump(trained_pipeline, file_obj)

In [16]:
from pathlib import Path
DATA_TRANSFORMATION_DIR = "data_transformation"
DATA_TRANSFORMATION_PIPELINE_DIR = "transformed_pipeline"
DATA_TRANSFORMATION_TRAIN_DIR = "train"
DATA_TRANSFORMATION_FILE_NAME = "consumer_complaint"
DATA_TRANSFORMATION_TEST_DIR = "test"
DATA_TRANSFORMATION_TEST_SIZE = 0.3

In [11]:
# dataframe: DataFrame = read_data()

test_size = 0.2 #data_tf_config.test_size
logger.info(f"Splitting dataset into train and test set using ratio: {1 - test_size}:{test_size}")
train_dataframe, test_dataframe = dataframe.randomSplit([1 - test_size, test_size])

logger.info(f"Train dataset has number of rows: [{train_dataframe.count()}] and "
            f"columns: [{len(train_dataframe.columns)}]")

# pipeline = get_data_transformation_pipeline()
transformed_pipeline = pipeline.fit(train_dataframe)

# Selecting required columns
required_columns = [schema.scaled_vector_input_features, schema.target_column]

transformed_trained_dataframe = transformed_pipeline.transform(train_dataframe)
transformed_trained_dataframe = transformed_trained_dataframe.select(required_columns)

transformed_test_dataframe = transformed_pipeline.transform(test_dataframe)
transformed_test_dataframe = transformed_test_dataframe.select(required_columns)



{'inputCols': ['company_response', 'consumer_consent_provided', 'submitted_via'], 'outputCols': ['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via']}


In [13]:
from  pathlib import Path

In [19]:
# Define file paths using pathlib
export_pipeline_dir = Path(DATA_TRANSFORMATION_PIPELINE_DIR)
print(export_pipeline_dir)
transformed_train_dir = Path(DATA_TRANSFORMATION_TRAIN_DIR)
transformed_test_dir = Path(DATA_TRANSFORMATION_TEST_DIR)

# Create directories
export_pipeline_dir.mkdir(parents=True, exist_ok=True)
transformed_train_dir.mkdir(parents=True, exist_ok=True)
transformed_test_dir.mkdir(parents=True, exist_ok=True)

# Define file paths for transformed data
transformed_train_data_file_path = transformed_train_dir / DATA_TRANSFORMATION_FILE_NAME
transformed_test_data_file_path = transformed_test_dir / DATA_TRANSFORMATION_FILE_NAME



transformed_pipeline


In [20]:
transformed_pipeline.write().save(str(export_pipeline_dir))


Py4JError: An error occurred while calling None.org.apache.spark.api.python.PythonFunction. Trace:
py4j.Py4JException: Constructor org.apache.spark.api.python.PythonFunction([class [B, class java.util.HashMap, class java.util.ArrayList, class java.lang.String, class java.lang.String, class java.util.ArrayList, class org.apache.spark.api.python.PythonAccumulatorV2]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:180)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:197)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:1589)



In [25]:
# Sample DataFrame
# creating imputer to fill null values
df = spark_session.read.parquet("/home/suyodhan/Documents/Data-Science-Project/Consumer-Complaint-Dispute-Prediction/consumer_artifact/data_validation/20230926_092906/accepted_data/consumer_complaint")
# df.printSchema()
derived_feature = DerivedFeatureGenerator(
                inputCols=schema.derived_input_features,
                outputCols=schema.derived_output_features
            )

# Define your pipeline stages (for demonstration purposes)

pipeline = Pipeline(stages=[derived_feature])

# Fit the pipeline to your DataFrame
trained_pipeline = pipeline.fit(df)

# Specify the directory where you want to save the trained pipeline
export_pipeline_dir = "/home/suyodhan/Documents/Data-Science-Project/Consumer-Complaint-Dispute-Prediction/research/transformed_pipeline"

# Save the trained pipeline to the specified directory
trained_pipeline.write().save(export_pipeline_dir)

Py4JError: An error occurred while calling None.org.apache.spark.api.python.PythonFunction. Trace:
py4j.Py4JException: Constructor org.apache.spark.api.python.PythonFunction([class [B, class java.util.HashMap, class java.util.ArrayList, class java.lang.String, class java.lang.String, class java.util.ArrayList, class org.apache.spark.api.python.PythonAccumulatorV2]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:180)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:197)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:1589)



In [3]:
from pathlib import Path
from dataclasses import  dataclass
@dataclass
class DataTransformationArtifact:
    transformed_train_file_path: Path
    exported_pipeline_file_path: Path
    transformed_test_file_path: Path


In [4]:
@dataclass 
class DataTransformationConfig:
    file_name: str
    export_pipeline_dir: Path
    transformed_train_dir: Path
    transformed_test_dir: Path
    test_size: float

In [5]:
from consumerComplaint.config.spark_manager import spark_session

In [6]:
dataframe = spark_session.read.parquet("/home/suyodhan/Documents/Data-Science-Project/Consumer-Complaint-Dispute-Prediction/consumer_artifact/data_validation/20230923_172536/accepted_data/consumer_complaint")

In [5]:
dataframe.show(10)

+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|             company|    company_response|consumer_consent_provided|consumer_disputed|       date_received|date_sent_to_company|               issue|             product|state|           sub_issue|submitted_via|timely|zip_code|
+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|TRANSUNION INTERM...|Closed with non-m...|     Consent not provided|              N/A|2023-06-05T12:00:...|2023-06-05T12:00:...|Improper use of y...|Credit reporting,...|   FL|Reporting company...|          Web|   Yes|   32832|
|TRANSUNION INTERM...|Closed with non-m...|         Consent provided|              N

In [6]:
dataframe.count()

10000

In [7]:
dataframe.select("company_response").show(5)
# dataframe.show(5)

+--------------------+
|    company_response|
+--------------------+
|Closed with non-m...|
|Closed with non-m...|
|Closed with non-m...|
|Closed with non-m...|
|Closed with non-m...|
+--------------------+
only showing top 5 rows



In [8]:
dataframe.select("consumer_consent_provided").show(5)

+-------------------------+
|consumer_consent_provided|
+-------------------------+
|     Consent not provided|
|         Consent provided|
|         Consent provided|
|     Consent not provided|
|     Consent not provided|
+-------------------------+
only showing top 5 rows



In [9]:
dataframe.select("submitted_via").show(5)

+-------------+
|submitted_via|
+-------------+
|          Web|
|          Web|
|          Web|
|          Web|
|          Web|
+-------------+
only showing top 5 rows



In [10]:
from pyspark.sql.functions import col, count, when, isnull

## Checking Null values

In [11]:
for column_name in dataframe.columns:
    dataframe.filter(col(column_name).isNull()).show()

+-------+----------------+-------------------------+-----------------+-------------+--------------------+-----+-------+-----+---------+-------------+------+--------+
|company|company_response|consumer_consent_provided|consumer_disputed|date_received|date_sent_to_company|issue|product|state|sub_issue|submitted_via|timely|zip_code|
+-------+----------------+-------------------------+-----------------+-------------+--------------------+-----+-------+-----+---------+-------------+------+--------+
+-------+----------------+-------------------------+-----------------+-------------+--------------------+-----+-------+-----+---------+-------------+------+--------+

+-------+----------------+-------------------------+-----------------+-------------+--------------------+-----+-------+-----+---------+-------------+------+--------+
|company|company_response|consumer_consent_provided|consumer_disputed|date_received|date_sent_to_company|issue|product|state|sub_issue|submitted_via|timely|zip_code|
+--

In [12]:
null_counts = dataframe.select([count(when(isnull(c), c)).alias(c) for c in dataframe.columns])

# Show the counts of null values
null_counts.show()

+-------+----------------+-------------------------+-----------------+-------------+--------------------+-----+-------+-----+---------+-------------+------+--------+
|company|company_response|consumer_consent_provided|consumer_disputed|date_received|date_sent_to_company|issue|product|state|sub_issue|submitted_via|timely|zip_code|
+-------+----------------+-------------------------+-----------------+-------------+--------------------+-----+-------+-----+---------+-------------+------+--------+
|      0|               0|                       82|                0|            0|                   0|    0|      0|   30|      495|            0|     0|       1|
+-------+----------------+-------------------------+-----------------+-------------+--------------------+-----+-------+-----+---------+-------------+------+--------+



In [13]:
from pyspark.ml.feature import StandardScaler, VectorAssembler, OneHotEncoder, StringIndexer, Imputer

In [14]:
dataframe.show(10)

+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|             company|    company_response|consumer_consent_provided|consumer_disputed|       date_received|date_sent_to_company|               issue|             product|state|           sub_issue|submitted_via|timely|zip_code|
+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|TRANSUNION INTERM...|Closed with non-m...|     Consent not provided|              N/A|2023-06-05T12:00:...|2023-06-05T12:00:...|Improper use of y...|Credit reporting,...|   FL|Reporting company...|          Web|   Yes|   32832|
|TRANSUNION INTERM...|Closed with non-m...|         Consent provided|              N

In [15]:
from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import Param, Params, TypeConverters, HasOutputCols, \
    HasInputCols
# Available in PySpark >= 2.3.0 
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.ml import Estimator
from pyspark.sql import DataFrame
from pyspark.sql.functions import desc
from pyspark.sql.functions import col, abs
from typing import List
from pyspark.sql.types import TimestampType, LongType
from consumerComplaint.logger import logger
from consumerComplaint.config.spark_manager import spark_session

In [16]:
60 * 60 * 24

86400

## DerivedFeatureGenerator

In [17]:


class DerivedFeatureGenerator(Transformer, HasInputCols, HasOutputCols,
                              DefaultParamsReadable, DefaultParamsWritable):

    @keyword_only
    def __init__(self, inputCols: List[str] = None, outputCols: List[str] = None, ):
        super(DerivedFeatureGenerator, self).__init__()
        kwargs = _input_kwargs
        # self._set(**kwargs)
        self.second_within_day = 86400 # 60 * 60 * 24 
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCols: List[str] = None, outputCols: List[str] = None, ):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def setInputCols(self, value: List[str]):
        """
        Sets the value of :py:attr:`inputCol`.
        """
        return self._set(inputCol=value)

    def setOutputCols(self, value: List[str]):
        """
        Sets the value of :py:attr:`outputCol`.
        """
        return self._set(outputCols=value)

    def _fit(self, dataframe: DataFrame):
        return self

    def _transform(self, dataframe: DataFrame):
        inputCols = self.getInputCols()

        for column in inputCols:
            dataframe = dataframe.withColumn(column,
                                             col(column).cast(TimestampType()))

        dataframe = dataframe.withColumn(self.getOutputCols()[0], abs(
            col(inputCols[1]).cast(LongType()) - col(inputCols[0]).cast(LongType())) / (
                                             self.second_within_day))
        return dataframe

In [18]:
from consumerComplaint.entity.schema import FinanceDataSchema
schema = FinanceDataSchema()

In [19]:
derived_input_features = schema.derived_input_features
print(derived_input_features)

derived_output_features = schema.derived_output_features
print(derived_output_features)

['date_sent_to_company', 'date_received']
['diff_in_days']


In [21]:
## Copy the dataframe
df_copy = dataframe.alias("df_copy")


In [22]:
# Assuming you have already created a SparkSession and DataFrame
from pyspark.ml import Pipeline

# Create an instance of the DerivedFeatureGenerator
derived_feature = DerivedFeatureGenerator(inputCols=schema.derived_input_features,
                                          outputCols=schema.derived_output_features)

# Create a Pipeline to apply the transformation
pipeline = Pipeline(stages=[derived_feature])

# Fit and transform your DataFrame using the Pipeline
transformed_df = pipeline.fit(df_copy).transform(df_copy)

# Show the resulting DataFrame with derived features
transformed_df.show(4)


+--------------------+--------------------+-------------------------+-----------------+-------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+------------+
|             company|    company_response|consumer_consent_provided|consumer_disputed|      date_received|date_sent_to_company|               issue|             product|state|           sub_issue|submitted_via|timely|zip_code|diff_in_days|
+--------------------+--------------------+-------------------------+-----------------+-------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+------------+
|TRANSUNION INTERM...|Closed with non-m...|     Consent not provided|              N/A|2023-06-05 22:30:00| 2023-06-05 22:30:00|Improper use of y...|Credit reporting,...|   FL|Reporting company...|          Web|   Yes|   32832|         0.0|
|TRANSUNION INTERM...|Closed with no

In [23]:
stages = []
stages.append(derived_feature)
stages

[DerivedFeatureGenerator_2846f98af8b3]

In [24]:
DATA_TRANSFORMATION_DIR = "data_transformation"
DATA_TRANSFORMATION_PIPELINE_DIR = "transformed_pipeline"
DATA_TRANSFORMATION_TRAIN_DIR = "train"
DATA_TRANSFORMATION_FILE_NAME = "finance_complaint"
DATA_TRANSFORMATION_TEST_DIR = "test"
DATA_TRANSFORMATION_TEST_SIZE = 0.3


In [25]:
# from consumerComplaint.config.pipeline.training import FinanceConfig 

import os , sys
from datetime import datetime
from pathlib import Path

from consumerComplaint.constants.training_pipeline_config import *
from consumerComplaint.constants import TIMESTAMP
from consumerComplaint.constants import *
from consumerComplaint.logger import logger
from consumerComplaint.exception import ConsumerComplaintException
from consumerComplaint.entity.metadata_entity import DataIngestionMetadata
from consumerComplaint.entity.config_entity import (DataIngestionConfig, 
                                                    TrainingPipelineConfig,
                                                    DataValidationConfig,
                                                    DataTransformationConfig)



class FinanceConfig:
    def __init__(self, pipeline_name=PIPELINE_NAME, timestamp=TIMESTAMP):
        """
        Name: FinanceConfig
        Description: Configuration class for the finance data processing pipeline.

        Organization: iNeuron Intelligence Private Limited

        Parameters:
        - pipeline_name: Name of the data processing pipeline.
        - timestamp: Timestamp associated with the configuration.

        Version: 1.0
        """
        self.timestamp = timestamp
        self.pipeline_name = pipeline_name
        self.pipeline_config = self.get_pipeline_config()

    def get_pipeline_config(self) -> TrainingPipelineConfig:
        """
        Name: get_pipeline_config
        Description: Retrieves the pipeline configuration information.

        Returns:
        - pipeline_config: Pipeline configuration details.

        On Failure: Writes an exception log and then raises an exception.

        Version: 1.0
        """
        try:
            artifact_dir = PIPELINE_ARTIFACT_DIR
            pipeline_config = TrainingPipelineConfig(pipeline_name=self.pipeline_name,
                                                     artifact_dir=artifact_dir)

            # Log pipeline configuration information.
            logger.info(f"Pipeline configuration: {pipeline_config}")

            return pipeline_config
        except Exception as e:
            # Log an exception message if an exception occurs during pipeline configuration retrieval.
            # logger.exception("Error during pipeline configuration retrieval.")
            raise ConsumerComplaintException(e, sys)


    def get_data_ingestion_config(self, from_date=DATA_INGESTION_MIN_START_DATE, to_date=None) -> DataIngestionConfig:
        """
        Name: get_data_ingestion_config
        Description: Retrieves data ingestion configuration information.

        Parameters:
        - from_date: Start date for data ingestion.
        - to_date: End date for data ingestion.

        Returns:
        - data_ingestion_config: Data ingestion configuration details.

        On Failure: Writes an exception log and then raises an exception.

        Version: 1.0
        """
        try:
            min_start_date = datetime.strptime(DATA_INGESTION_MIN_START_DATE, "%Y-%m-%d")
            from_date_obj = datetime.strptime(from_date, "%Y-%m-%d")
            
            # Adjust from_date if it is earlier than the minimum start date
            if from_date_obj < min_start_date:
                from_date = DATA_INGESTION_MIN_START_DATE
            
            if to_date is None:
                to_date = datetime.now().strftime("%Y-%m-%d")

            # Define master directory for data ingestion
            data_ingestion_master_dir = Path(self.pipeline_config.artifact_dir) / DATA_INGESTION_DIR

            # Create a time-based directory for each run
            data_ingestion_dir = data_ingestion_master_dir / self.timestamp

            metadata_file_path = data_ingestion_master_dir / DATA_INGESTION_METADATA_FILE_NAME

            data_ingestion_metadata = DataIngestionMetadata(metadata_file_path=metadata_file_path)

            if data_ingestion_metadata.is_metadata_file_present:
                metadata_info = data_ingestion_metadata.get_metadata_info()
                from_date = metadata_info.to_date

            data_ingestion_config = DataIngestionConfig(
                from_date=from_date,
                to_date=to_date,
                data_ingestion_dir=data_ingestion_dir,
                download_dir=os.path.join(data_ingestion_dir, DATA_INGESTION_DOWNLOADED_DATA_DIR),
                file_name=DATA_INGESTION_FILE_NAME,
                feature_store_dir=os.path.join(data_ingestion_master_dir, DATA_INGESTION_FEATURE_STORE_DIR),
                failed_dir=os.path.join(data_ingestion_dir, DATA_INGESTION_FAILED_DIR),
                metadata_file_path=metadata_file_path,
                datasource_url=DATA_INGESTION_DATA_SOURCE_URL
            )
            
            # Log data ingestion configuration information.
            logger.info(f"Data ingestion config: {data_ingestion_config}")
            
            return data_ingestion_config

        except Exception as e:
            # Log an exception message if an exception occurs during data ingestion configuration retrieval.
            # logger.exception("Error during data ingestion configuration retrieval.")
            raise ConsumerComplaintException(e, sys)


    def get_data_validation_config(self) -> DataValidationConfig:
        """
        Name: get_data_validation_config
        Description: Retrieves data validation configuration information.

        Returns:
        - data_validation_config: Data validation configuration details.

        On Failure: Writes an exception log and then raises an exception.

        Version: 1.0
        """
        try:
            data_validation_master_dir = Path(self.pipeline_config.artifact_dir) / DATA_VALIDATION_DIR
            data_validation_dir = data_validation_master_dir / self.timestamp

            accepted_data_dir = Path(data_validation_dir) / DATA_VALIDATION_ACCEPTED_DATA_DIR
            rejected_data_dir = Path(data_validation_dir) / DATA_VALIDATION_REJECTED_DATA_DIR

            data_validation_config = DataValidationConfig(
                accepted_data_dir=accepted_data_dir,
                rejected_data_dir=rejected_data_dir,
                file_name=DATA_VALIDATION_FILE_NAME
            )

            # Log data validation configuration information.
            logger.info(f"Data validation config: {data_validation_config}")

            return data_validation_config
        except Exception as e:
            # Log an exception message if an exception occurs during data validation configuration retrieval.
            logger.exception("Error during data validation configuration retrieval.")
            raise ConsumerComplaintException(e, sys)

    def get_data_transformation_config(self) -> DataTransformationConfig:
        try:
            data_transformation_dir = Path(self.pipeline_config.artifact_dir) / DATA_TRANSFORMATION_DIR / self.timestamp

            transformed_train_data_dir = data_transformation_dir / DATA_TRANSFORMATION_TRAIN_DIR

            transformed_test_data_dir = data_transformation_dir / DATA_TRANSFORMATION_TEST_DIR

            export_pipeline_dir = data_transformation_dir / DATA_TRANSFORMATION_PIPELINE_DIR
            
            data_transformation_config = DataTransformationConfig(
                export_pipeline_dir=export_pipeline_dir,
                transformed_test_dir=transformed_test_data_dir,
                transformed_train_dir=transformed_train_data_dir,
                file_name=DATA_TRANSFORMATION_FILE_NAME,
                test_size=DATA_TRANSFORMATION_TEST_SIZE,
            )

            logger.info(f"Data transformation config: {data_transformation_config}")
            return data_transformation_config
        except Exception as e:
            raise ConsumerComplaintException(e, sys)

In [26]:
finance = FinanceConfig()
finance.get_data_transformation_config()

DataTransformationConfig(file_name='finance_complaint', export_pipeline_dir=PosixPath('/home/suyodhan/Documents/Data-Science-Project/Consumer-Complaint-Dispute-Prediction/research/consumer_artifact/data_transformation/20230924_115020/transformed_pipeline'), transformed_train_dir=PosixPath('/home/suyodhan/Documents/Data-Science-Project/Consumer-Complaint-Dispute-Prediction/research/consumer_artifact/data_transformation/20230924_115020/train'), transformed_test_dir=PosixPath('/home/suyodhan/Documents/Data-Science-Project/Consumer-Complaint-Dispute-Prediction/research/consumer_artifact/data_transformation/20230924_115020/test'), test_size=0.3)

In [ ]:
# ## SAmple examples
# data = [(1, "2023-09-20 10:00:00", "2023-09-20 12:00:00"),
#         (2, "2023-09-20 08:00:00", "2023-09-20 13:00:00")]
# columns = ["id", "timestamp1", "timestamp2"]
# df = spark_session.createDataFrame(data, columns)

# derived_feature_generator = DerivedFeatureGenerator(inputCols=["timestamp1", "timestamp2"], outputCols=["time_diff"])
# result_df = derived_feature_generator.transform(df)

# result_df.show()

## Null Values

In [27]:
null_counts = dataframe.select([count(when(isnull(c), c)).alias(c) for c in dataframe.columns])

# Show the counts of null values
null_counts.show()

+-------+----------------+-------------------------+-----------------+-------------+--------------------+-----+-------+-----+---------+-------------+------+--------+
|company|company_response|consumer_consent_provided|consumer_disputed|date_received|date_sent_to_company|issue|product|state|sub_issue|submitted_via|timely|zip_code|
+-------+----------------+-------------------------+-----------------+-------------+--------------------+-----+-------+-----+---------+-------------+------+--------+
|      0|               0|                       82|                0|            0|                   0|    0|      0|   30|      495|            0|     0|       1|
+-------+----------------+-------------------------+-----------------+-------------+--------------------+-----+-------+-----+---------+-------------+------+--------+



In [ ]:
# null_counts = dataframe.select([count(when(isnull(c), c)).alias(c) for c in dataframe.columns])

# # Show the counts of null values
# null_counts.show()

In [40]:
## consumer_consent_provided, state, sub_issue, zip_code
from consumerComplaint.entity.schema import FinanceDataSchema
schema = FinanceDataSchema()

In [44]:
dataframe.select('consumer_consent_provided').distinct().show()

+-------------------------+
|consumer_consent_provided|
+-------------------------+
|        Consent withdrawn|
|                    Other|
|         Consent provided|
|     Consent not provided|
|                      N/A|
|                     NULL|
+-------------------------+



In [45]:
dataframe.select('company_response').distinct().show()

+--------------------+
|    company_response|
+--------------------+
|   Untimely response|
|Closed with non-m...|
|         In progress|
|Closed with monet...|
|Closed with expla...|
+--------------------+



In [46]:
dataframe.select('state').distinct().show()

+-----+
|state|
+-----+
|   AZ|
|   SC|
|   LA|
|   MN|
|   NJ|
|   DC|
|   OR|
|   VA|
|   RI|
|   KY|
|   WY|
|   NH|
|   MI|
|   NV|
|   WI|
|   ID|
|   CA|
|   NE|
|   CT|
|   MT|
+-----+
only showing top 20 rows



In [47]:
dataframe.select('sub_issue').distinct().show()

+--------------------+
|           sub_issue|
+--------------------+
|      Banking errors|
|      Privacy issues|
|Transaction was n...|
|Problem getting y...|
|Problem when atte...|
|Old information r...|
|Account opened as...|
|Account status in...|
|Deposits or withd...|
|Information is in...|
|Talked to a third...|
|Told you not to r...|
|Confusing or miss...|
|       Credit denial|
|Company closed yo...|
|Credit card compa...|
|Problem with rewa...|
|Impersonated atto...|
|Bounced checks or...|
|Card company isn'...|
+--------------------+
only showing top 20 rows



In [48]:
dataframe.select('zip_code').distinct().show()

+--------+
|zip_code|
+--------+
|   91910|
|   33174|
|   32812|
|   19132|
|   178XX|
|   08648|
|   85022|
|   29210|
|   75602|
|   06518|
|   75007|
|   11563|
|   803XX|
|   55445|
|   11722|
|   60081|
|   43085|
|   77339|
|   068XX|
|   94102|
+--------+
only showing top 20 rows



In [59]:
# Filter rows with missing values in the "zip_code" column
rows_with_missing_zip_code = transformed_df.filter(transformed_df["zip_code"].isNull())

# Show the rows with missing "zip_code" values
rows_with_missing_zip_code.show()


+-------------+--------------------+-------------------------+-----------------+-------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+------------+---------------+
|      company|    company_response|consumer_consent_provided|consumer_disputed|      date_received|date_sent_to_company|               issue|             product|state|           sub_issue|submitted_via|timely|zip_code|diff_in_days|im_diff_in_days|
+-------------+--------------------+-------------------------+-----------------+-------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+------------+---------------+
|EQUIFAX, INC.|Closed with non-m...|     Consent not provided|              N/A|2023-06-05 22:30:00| 2023-06-05 22:30:00|Improper use of y...|Credit reporting,...|   CA|Reporting company...|          Web|   Yes|    NULL|         0.0|            0.0|


In [61]:
stages = []

In [62]:
# Create the Imputer stage
imputer = Imputer(
    inputCols=schema.numerical_columns,  # Replace with your input columns
    outputCols=schema.im_numerical_columns,  # Replace with your output columns
)

stages.append(imputer)


# Create a Pipeline with the Imputer stage
# pipeline = Pipeline(stages=[imputer])

# Fit and transform your DataFrame using the Pipeline
# transformed_df = pipeline.fit(df_copy).transform(df_copy)

# Show the resulting DataFrame with imputed values
# pipeline

In [ ]:
frequency_imputer = FrequencyImputer(inputCols=self.schema.one_hot_encoding_features,
                                                 outputCols=self.schema.im_one_hot_encoding_features)

In [63]:
schema.one_hot_encoding_features

['company_response', 'consumer_consent_provided', 'submitted_via']

In [65]:
dataframe.columns

['company',
 'company_response',
 'consumer_consent_provided',
 'consumer_disputed',
 'date_received',
 'date_sent_to_company',
 'issue',
 'product',
 'state',
 'sub_issue',
 'submitted_via',
 'timely',
 'zip_code']

In [64]:
schema.string_indexer_one_hot_features

['si_company_response', 'si_consumer_consent_provided', 'si_submitted_via']

In [57]:
stages = pipeline.getStages()
# Iterate through the stages and show their details
for stage in stages:
    print("Stage Name:", stage.uid)
    print("Stage Type:", type(stage).__name__)
    if hasattr(stage, 'getInputCols'):
        print("Input Columns:", stage.getInputCols())
    if hasattr(stage, 'getOutputCols'):
        print("Output Columns:", stage.getOutputCols())
    print("\n")

Stage Name: Imputer_1b40d48fd9b3
Stage Type: Imputer
Input Columns: ['company_response', 'consumer_consent_provided', 'state', 'sub_issue', 'zip_code']
Output Columns: ['im_company_response', 'im_consumer_consent_provided', 'im_state', 'im_sub_issue', 'im_zip_code']




In [ ]:
df_copy = function

In [60]:
schema.numerical_columns

['diff_in_days']

In [58]:
dataframe.show(4)

+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|             company|    company_response|consumer_consent_provided|consumer_disputed|       date_received|date_sent_to_company|               issue|             product|state|           sub_issue|submitted_via|timely|zip_code|
+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|TRANSUNION INTERM...|Closed with non-m...|     Consent not provided|              N/A|2023-06-05T12:00:...|2023-06-05T12:00:...|Improper use of y...|Credit reporting,...|   FL|Reporting company...|          Web|   Yes|   32832|
|TRANSUNION INTERM...|Closed with non-m...|         Consent provided|              N

In [72]:
import pandas as pd

# Assuming 'dataframe' is your DataFrame and 'column_name' is the name of the column you want to print
column_data = dataframe.select("issue").toPandas()["issue"]

for value in column_data:
    print(value)


Improper use of your report
Improper use of your report
Improper use of your report
Improper use of your report
Incorrect information on your report
Problem with a credit reporting company's investigation into an existing problem
Improper use of your report
Closing an account
Improper use of your report
Incorrect information on your report
Improper use of your report
Problem with a credit reporting company's investigation into an existing problem
Problem with a credit reporting company's investigation into an existing problem
Incorrect information on your report
Improper use of your report
Incorrect information on your report
Problem with a credit reporting company's investigation into an existing problem
Improper use of your report
Written notification about debt
Improper use of your report
Improper use of your report
Incorrect information on your report
Improper use of your report
Incorrect information on your report
Problem with a credit reporting company's investigation into an exi

In [73]:
from pyspark.sql.functions import col

# Assuming 'dataframe' is your DataFrame
for column_name in dataframe.columns:
    unique_count = dataframe.select(column_name).distinct().count()
    print(f"Column '{column_name}' has {unique_count} unique values.")


Column 'company' has 532 unique values.
Column 'company_response' has 5 unique values.
Column 'consumer_consent_provided' has 6 unique values.
Column 'consumer_disputed' has 1 unique values.
Column 'date_received' has 31 unique values.
Column 'date_sent_to_company' has 75 unique values.
Column 'issue' has 64 unique values.
Column 'product' has 9 unique values.
Column 'state' has 57 unique values.
Column 'sub_issue' has 137 unique values.
Column 'submitted_via' has 5 unique values.
Column 'timely' has 2 unique values.
Column 'zip_code' has 4200 unique values.


#### Differnce between sent and recived date

In [35]:
numerical_columns = schema.numerical_columns
print(numerical_columns)

im_numerical_columns = schema.im_numerical_columns
print(im_numerical_columns)

['diff_in_days']
['im_diff_in_days']


In [13]:
df = spark_session.read.parquet("/home/suyodhan/Documents/Data-Science-Project/Consumer-Complaint-Dispute-Prediction/consumer_artifact/data_validation/20230926_114259/accepted_data/consumer_complaint")

In [8]:
stages = []

In [9]:
derived_feature = DerivedFeatureGenerator(inputCols=schema.derived_input_features,
                                                      outputCols=schema.derived_output_features)
stages.append(derived_feature)

In [10]:
stages

[DerivedFeatureGenerator_f5f68b042e90]

In [11]:
pipeline = Pipeline(
    stages=stages )

In [21]:
transformed = pipeline.fit(df)

In [22]:
transformed.write().save("/home/suyodhan/Documents/Data-Science-Project/Consumer-Complaint-Dispute-Prediction/research/transformed_pipeline")
# Assuming 'transformed' is a valid DataFrame
# Assuming 'transformed' is a valid DataFrame
# transformed.write.mode("overwrite").parquet("/home/suyodhan/Documents/Data-Science-Project/Consumer-Complaint-Dispute-Prediction/research/transformed_pipeline")


Py4JError: An error occurred while calling None.org.apache.spark.api.python.PythonFunction. Trace:
py4j.Py4JException: Constructor org.apache.spark.api.python.PythonFunction([class [B, class java.util.HashMap, class java.util.ArrayList, class java.lang.String, class java.lang.String, class java.util.ArrayList, class org.apache.spark.api.python.PythonAccumulatorV2]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:180)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:197)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:1589)



In [44]:
imputer = Imputer(inputCols=schema.numerical_columns,
                              outputCols=schema.im_numerical_columns)
stages.append(imputer)

In [45]:
dataframe.columns

['company',
 'company_response',
 'consumer_consent_provided',
 'consumer_disputed',
 'date_received',
 'date_sent_to_company',
 'issue',
 'product',
 'state',
 'sub_issue',
 'submitted_via',
 'timely',
 'zip_code']

In [46]:
schema.one_hot_encoding_features

['company_response', 'consumer_consent_provided', 'submitted_via']

In [47]:
schema.im_one_hot_encoding_features

['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via']

In [48]:
## CATEGORICAL FEATURE 
frequency_imputer = FrequencyImputer(inputCols=schema.one_hot_encoding_features,
                                                 outputCols=schema.im_one_hot_encoding_features)
stages.append(frequency_imputer)

{'inputCols': ['company_response', 'consumer_consent_provided', 'submitted_via'], 'outputCols': ['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via']}


In [49]:
schema.string_indexer_one_hot_features

['si_company_response', 'si_consumer_consent_provided', 'si_submitted_via']

In [50]:
for im_one_hot_feature, string_indexer_col in zip(schema.im_one_hot_encoding_features,
                                                              schema.string_indexer_one_hot_features):
                string_indexer = StringIndexer(inputCol=im_one_hot_feature, outputCol=string_indexer_col)
                stages.append(string_indexer)


In [51]:
one_hot_encoder = OneHotEncoder(inputCols=schema.string_indexer_one_hot_features,
                                            outputCols=schema.tf_one_hot_encoding_features)
stages.append(one_hot_encoder)

In [52]:
schema.tfidf_features

['issue']

In [53]:
dataframe.select(['issue']).show(10)

+--------------------+
|               issue|
+--------------------+
|Improper use of y...|
|Improper use of y...|
|Improper use of y...|
|Improper use of y...|
|Incorrect informa...|
|Problem with a cr...|
|Improper use of y...|
|  Closing an account|
|Improper use of y...|
|Incorrect informa...|
+--------------------+
only showing top 10 rows



In [54]:
dataframe.select(schema.tfidf_features[0]).show(10)

+--------------------+
|               issue|
+--------------------+
|Improper use of y...|
|Improper use of y...|
|Improper use of y...|
|Improper use of y...|
|Incorrect informa...|
|Problem with a cr...|
|Improper use of y...|
|  Closing an account|
|Improper use of y...|
|Incorrect informa...|
+--------------------+
only showing top 10 rows



In [55]:
tokenizer = Tokenizer(inputCol=schema.tfidf_features[0], outputCol="words")
stages.append(tokenizer)

In [56]:
hashing_tf = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="rawFeatures", numFeatures=40)
stages.append(hashing_tf)

In [57]:
idf = IDF(inputCol=hashing_tf.getOutputCol(), outputCol=schema.tf_tfidf_features[0])
stages.append(idf)


In [58]:
vector_assembler = VectorAssembler(inputCols=schema.input_features,
                                    outputCol=schema.vector_assembler_output)

stages.append(vector_assembler)

In [59]:
standard_scaler = StandardScaler(inputCol=schema.vector_assembler_output,
                                             outputCol=schema.scaled_vector_input_features)
stages.append(standard_scaler)



In [60]:
pipeline = Pipeline(
    stages=stages )

In [61]:
train_dataframe, test_dataframe = dataframe.randomSplit([1 - 0.2, 0.2])

In [62]:
transformed_pipeline = pipeline.fit(train_dataframe)


{'inputCols': ['company_response', 'consumer_consent_provided', 'submitted_via'], 'outputCols': ['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via']}


In [64]:
fitted_stages = transformed_pipeline.stages
fitted_stages

[DerivedFeatureGenerator_6877f842acd1,
 ImputerModel: uid=Imputer_98b8ea192f85, strategy=mean, missingValue=NaN, numInputCols=1, numOutputCols=1,
 FrequencyImputerModel_24762a4bdcfa,
 StringIndexerModel: uid=StringIndexer_bc166790457b, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_a2b1211ca429, handleInvalid=error,
 StringIndexerModel: uid=StringIndexer_b336d5fd52d7, handleInvalid=error,
 OneHotEncoderModel: uid=OneHotEncoder_1dddd02663e2, dropLast=true, handleInvalid=error, numInputCols=3, numOutputCols=3,
 Tokenizer_7414b18a0dc2,
 HashingTF_80d001f38c45,
 IDFModel: uid=IDF_27a09182dc1e, numDocs=7989, numFeatures=40,
 VectorAssembler_885d10a47306,
 StandardScalerModel: uid=StandardScaler_c385513582a7, numFeatures=53, withMean=false, withStd=true]

In [65]:
stage_params = [stage.extractParamMap() for stage in fitted_stages]
stage_params

[{Param(parent='DerivedFeatureGenerator_6877f842acd1', name='inputCols', doc='input column names.'): ['date_sent_to_company',
   'date_received'],
  Param(parent='DerivedFeatureGenerator_6877f842acd1', name='outputCols', doc='output column names.'): ['diff_in_days']},
 {Param(parent='Imputer_98b8ea192f85', name='missingValue', doc='The placeholder for the missing values. All occurrences of missingValue will be imputed.'): nan,
  Param(parent='Imputer_98b8ea192f85', name='outputCol', doc='output column name.'): 'Imputer_98b8ea192f85__output',
  Param(parent='Imputer_98b8ea192f85', name='relativeError', doc='the relative target precision for the approximate quantile algorithm. Must be in the range [0, 1]'): 0.001,
  Param(parent='Imputer_98b8ea192f85', name='strategy', doc='strategy for imputation. If mean, then replace missing values using the mean value of the feature. If median, then replace missing values using the median value of the feature. If mode, then replace missing using the 

In [66]:
# Access the stages (transformers and estimators) in the pipeline
fitted_stages = transformed_pipeline.stages

# Access the parameters of each stage in the fitted pipeline
stage_params = [stage.extractParamMap() for stage in fitted_stages]

# Print the names of stages and their parameters
for i, stage in enumerate(fitted_stages):
    print(f"Stage {i + 1}: {stage.__class__.__name__}")
    for param, value in stage_params[i].items():
        print(f"  {param.name}: {value}")

# Example: Accessing a specific stage's parameters
# Assuming the first stage is a VectorAssembler
vector_assembler_params = stage_params[0]
print("VectorAssembler InputCols:", vector_assembler_params[vector_assembler.getParam("inputCols")])

Stage 1: DerivedFeatureGenerator
  inputCols: ['date_sent_to_company', 'date_received']
  outputCols: ['diff_in_days']
Stage 2: ImputerModel
  missingValue: nan
  outputCol: Imputer_98b8ea192f85__output
  relativeError: 0.001
  strategy: mean
  inputCols: ['diff_in_days']
  outputCols: ['im_diff_in_days']
Stage 3: FrequencyImputerModel
  topCategorys: ['Closed with explanation', 'Consent provided', 'Web']
  inputCols: ['company_response', 'consumer_consent_provided', 'submitted_via']
  outputCols: ['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via']
Stage 4: StringIndexerModel
  handleInvalid: error
  outputCol: si_company_response
  stringOrderType: frequencyDesc
  inputCol: im_company_response
Stage 5: StringIndexerModel
  handleInvalid: error
  outputCol: si_consumer_consent_provided
  stringOrderType: frequencyDesc
  inputCol: im_consumer_consent_provided
Stage 6: StringIndexerModel
  handleInvalid: error
  outputCol: si_submitted_via
  stringOrderType: frequ

KeyError: Param(parent='VectorAssembler_885d10a47306', name='inputCols', doc='input column names.')

In [72]:
# selecting required columns
required_columns = [schema.scaled_vector_input_features, 'consumer_disputed']




In [73]:
required_columns

['scaled_input_features', 'consumer_disputed']

In [74]:
transformed_trained_dataframe = transformed_pipeline.transform(train_dataframe)


In [76]:
transformed_trained_dataframe

DataFrame[scaled_input_features: vector, consumer_disputed: string]

In [75]:
transformed_trained_dataframe = transformed_trained_dataframe.select(required_columns)



In [77]:
transformed_trained_dataframe

DataFrame[scaled_input_features: vector, consumer_disputed: string]

In [78]:
transformed_test_dataframe = transformed_pipeline.transform(test_dataframe)
transformed_test_dataframe = transformed_test_dataframe.select(required_columns)

export_pipeline_file_path = "data_transformation"

In [79]:
# creating required directory
os.makedirs(export_pipeline_file_path, exist_ok=True)
os.makedirs("data_transformation/test", exist_ok=True)
os.makedirs("data_transformation/train", exist_ok=True)
transformed_train_data_file_path = os.path.join("data_transformation/train",
                                                "prashnt"
                                                )
transformed_test_data_file_path = os.path.join("data_transformation/test",
                                                "prashnt"
                                                           )

In [80]:
transformed_pipeline.save(export_pipeline_file_path)

Py4JError: An error occurred while calling None.org.apache.spark.api.python.PythonFunction. Trace:
py4j.Py4JException: Constructor org.apache.spark.api.python.PythonFunction([class [B, class java.util.HashMap, class java.util.ArrayList, class java.lang.String, class java.lang.String, class java.util.ArrayList, class org.apache.spark.api.python.PythonAccumulatorV2]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:180)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:197)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:1589)



In [84]:
# Assuming your DataFrame is named 'dataframe' and the target column is 'consumer_disputed'
from pyspark.sql.functions import col

# Count the unique values and their counts
unique_counts = dataframe.groupBy("consumer_disputed").count().orderBy("count", ascending=False)

# Show the unique values and their counts
unique_counts.show()


+-----------------+-----+
|consumer_disputed|count|
+-----------------+-----+
|              N/A|10000|
+-----------------+-----+



In [85]:
df = spark_session.read.parquet("/home/suyodhan/Documents/Data-Science-Project/Consumer-Complaint-Dispute-Prediction/consumer_artifact/data_validation/20230923_172536/accepted_data/consumer_complaint/")

In [86]:
df.show(5)

+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|             company|    company_response|consumer_consent_provided|consumer_disputed|       date_received|date_sent_to_company|               issue|             product|state|           sub_issue|submitted_via|timely|zip_code|
+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|TRANSUNION INTERM...|Closed with non-m...|     Consent not provided|              N/A|2023-06-05T12:00:...|2023-06-05T12:00:...|Improper use of y...|Credit reporting,...|   FL|Reporting company...|          Web|   Yes|   32832|
|TRANSUNION INTERM...|Closed with non-m...|         Consent provided|              N

In [71]:
# # Create a Pipeline to apply the transformation
# pipeline = Pipeline(stages=[derived_feature, imputer])

# # Fit and transform your DataFrame using the Pipeline
# transformed_df = pipeline.fit(df_copy).transform(df_copy)

# # Show the resulting DataFrame with derived features
# transformed_df.show(4)

+--------------------+--------------------+-------------------------+-----------------+-------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+------------+---------------+
|             company|    company_response|consumer_consent_provided|consumer_disputed|      date_received|date_sent_to_company|               issue|             product|state|           sub_issue|submitted_via|timely|zip_code|diff_in_days|im_diff_in_days|
+--------------------+--------------------+-------------------------+-----------------+-------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+------------+---------------+
|TRANSUNION INTERM...|Closed with non-m...|     Consent not provided|              N/A|2023-06-05 22:30:00| 2023-06-05 22:30:00|Improper use of y...|Credit reporting,...|   FL|Reporting company...|          Web|   Yes|   32832|  

In [34]:
dataframe.show(2)

+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|             company|    company_response|consumer_consent_provided|consumer_disputed|       date_received|date_sent_to_company|               issue|             product|state|           sub_issue|submitted_via|timely|zip_code|
+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|TRANSUNION INTERM...|Closed with non-m...|     Consent not provided|              N/A|2023-06-05T12:00:...|2023-06-05T12:00:...|Improper use of y...|Credit reporting,...|   FL|Reporting company...|          Web|   Yes|   32832|
|TRANSUNION INTERM...|Closed with non-m...|         Consent provided|              N

In [35]:
dataframe.select('consumer_consent_provided').show(5)

+-------------------------+
|consumer_consent_provided|
+-------------------------+
|     Consent not provided|
|         Consent provided|
|         Consent provided|
|     Consent not provided|
|     Consent not provided|
+-------------------------+
only showing top 5 rows



In [36]:
dataframe.select('consumer_consent_provided').distinct().show()

+-------------------------+
|consumer_consent_provided|
+-------------------------+
|        Consent withdrawn|
|                    Other|
|         Consent provided|
|     Consent not provided|
|                      N/A|
|                     NULL|
+-------------------------+



In [90]:
dataframe.show(20)

+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|             company|    company_response|consumer_consent_provided|consumer_disputed|       date_received|date_sent_to_company|               issue|             product|state|           sub_issue|submitted_via|timely|zip_code|
+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|TRANSUNION INTERM...|Closed with non-m...|     Consent not provided|              N/A|2023-06-05T12:00:...|2023-06-05T12:00:...|Improper use of y...|Credit reporting,...|   FL|Reporting company...|          Web|   Yes|   32832|
|TRANSUNION INTERM...|Closed with non-m...|         Consent provided|              N

In [3]:
# Read the JSON file into a Spark DataFrame
json_file_path = "/home/suyodhan/Documents/Data-Science-Project/dataset/complaints.json"
json_df = spark_session.read.json(json_file_path).limit(1000)

# Save the DataFrame as a Parquet file
# parquet_output_path = "/home/suyodhan/Documents/Data-Science-Project/dataset/parquet_file.parquet"
# json_df.write.parquet(parquet_output_path)

In [5]:
json_df.show()

+---------------+--------------------+-----------------------+--------------------+------------+-----------------------+-------------------------+-----------------+-------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+-------------+-------------+------+--------+
|_corrupt_record|             company|company_public_response|    company_response|complaint_id|complaint_what_happened|consumer_consent_provided|consumer_disputed|date_received|date_sent_to_company|               issue|             product|state|           sub_issue|         sub_product|submitted_via|         tags|timely|zip_code|
+---------------+--------------------+-----------------------+--------------------+------------+-----------------------+-------------------------+-----------------+-------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+-------------+-------------+------+--------

In [7]:
dataframe.show()

+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|             company|    company_response|consumer_consent_provided|consumer_disputed|       date_received|date_sent_to_company|               issue|             product|state|           sub_issue|submitted_via|timely|zip_code|
+--------------------+--------------------+-------------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-------------+------+--------+
|TRANSUNION INTERM...|Closed with non-m...|     Consent not provided|              N/A|2023-06-05T12:00:...|2023-06-05T12:00:...|Improper use of y...|Credit reporting,...|   FL|Reporting company...|          Web|   Yes|   32832|
|TRANSUNION INTERM...|Closed with non-m...|         Consent provided|              N